# В конце этого семинара - домашка!

# PyTorch

## __Tensors__

Одно из основных понятий в PyTorch -- это __Tenosor__. 

https://pytorch.org/docs/master/tensors.html

__Tensor__ -- это такой же массив, как и в __numpy.array__, размерность и тип данных которого мы можем задать. Tensor в отличие от numpy.array может вычисляться на __GPU__.

In [1]:
import numpy as np
import torch

In [2]:
N = 100
D_in = 50

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

x = np.random.randn(N, D_in)
x_torch = torch.randn(N, D_in, device=device, dtype=dtype)

In [3]:
x

array([[-0.03590312, -1.27655174, -0.44052684, ..., -0.43147624,
        -0.25771107,  0.21286299],
       [ 1.91601787,  0.74759333, -0.7333231 , ..., -0.73324062,
         0.52327749, -0.87981242],
       [ 0.7147454 ,  0.75085662,  1.02635125, ..., -0.15367271,
        -0.81197602,  0.63982388],
       ...,
       [ 0.04861756,  0.14869406, -1.0271621 , ...,  1.8787875 ,
        -1.0315354 , -1.01548184],
       [ 0.86040269,  0.50469198,  0.46516521, ..., -0.19889741,
         1.18849519,  1.0904733 ],
       [-1.60123939,  0.95751774, -0.31545188, ..., -0.39356475,
        -0.81519704, -1.56058633]])

In [4]:
x_torch

tensor([[ 1.6057,  0.7498,  0.3816,  ..., -0.4781,  0.2219,  0.9126],
        [-1.0428, -0.0726, -0.8711,  ...,  0.3939,  0.4245,  2.5646],
        [-1.7572,  1.1117, -0.7665,  ..., -0.8055, -0.0551,  1.0070],
        ...,
        [-0.5492, -0.3966,  0.3215,  ...,  0.0224,  0.9706, -0.9049],
        [ 0.3233, -1.0059, -0.5393,  ...,  0.2989,  0.8244, -1.2695],
        [ 0.0388, -0.2677, -0.7803,  ...,  0.1279,  0.0583,  0.9630]])

In [5]:
x_torch = torch.Tensor(np.ones((N, D_in)))
x_torch

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [6]:
x_torch = torch.FloatTensor([1, 2, 3])
x_torch

tensor([1., 2., 3.])

In [7]:
x1 = torch.IntTensor([1, 2, 3])
x2 = torch.FloatTensor([3, 4, 5])

In [8]:
print(x1)
print(x2)

tensor([1, 2, 3], dtype=torch.int32)
tensor([3., 4., 5.])


В PyTorch можно найти много операций, которые похожи на то, что есть в numpy :
```
- torch.add (np.add) -> сложение тензоров (поэлементное)
- torch.sub (np.subtract) -> вычитание (поэлементное)
- torch.mul (np.multiply) -> умнажение скаляров / матриц (поэлементное)
- torch.mm (np.matmul) -> перемножение матриц
- torch.ones (np.ones) -> создание тензора из единиц
```

In [15]:
# Давайте попробуем вышепересчисленные операции

In [9]:
x1 = torch.FloatTensor([[1, 2, 3], [4, 5, 6]])

In [10]:
x2 = torch.FloatTensor([[7, 8], [9, 1], [2, 3]])

In [11]:
out = torch.mm(x1, x2)
out

tensor([[31., 19.],
        [85., 55.]])

```
- torch.view (np.reshape) -> изменения порядка элементов в тензоре, не путать с транспонированием.
```

## Dynamic Computational Graph

После того, как были реализованы архитектура модели и весь процес обучения и валидация сети, при запуске кода в PyTorch происходят следующие этапы:

1. Строится вычислительный граф (направленный ациклический граф), где каждое ребро, ведущее к дргуому узлу, -- это тензор, а узел - это выполнение операции над данным тензором.

<img src="./images/Graph.png" alt="Drawing" style="width: 300px;"/>

Реализуем двухслойную сеть для задачи регрессии. И граф для такой архитектуры бдует выглядить следующим образом:

<img src="./images/RegGraph.png" alt="Drawing" />

In [3]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

In [18]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    h_1 = x.mm(w1)
    h_relu = h_1.clamp(min=0)
    out = h_relu.mm(w2)

    loss = (out - y).pow(2).sum().item() #MSE

    dloss_out = 2 * (out - y)  # первая производная по лоссу

    grad_w2 = h_relu.t().mm(dloss_out)
    grad_hrelu = dloss_out.mm(w2.t())
    grad_hrelu[h_1 < 0] = 0
    grad_w1 = x.t().mm(grad_hrelu)
 
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    if t % 100 == 99:
        print('Loss on iteration {} = {}'.format(t, loss))
    

Loss on iteration 99 = 68.03770446777344
Loss on iteration 199 = 67.95539855957031
Loss on iteration 299 = 67.87477111816406
Loss on iteration 399 = 67.79576110839844
Loss on iteration 499 = 67.71833801269531


In [5]:
loss

137.5653076171875

## Autograd

2. Еще одно фундаментальное понятие и важный элемент при построении графа -- это __Autograd__ -- автоматическое дифференцирование.

Для того чтобы с помощью стохастического градиентного спуска обновить обучаемые параметры сети, нужно посчитать градиенты. И как известно, обновление весов, которые учавтсвуют в нескольких операциях, происходит по `правилу дифференцирования сложной функции` (цепное правило или __chain rule__).

<img src="./images/RegChainRule.png" alt="Drawing" />

То есть (1) вычислительный граф позволяет определить последовательность операций, а (2) автоматическое дифференцирование посчитать нужные градиенты.

Если бы `Autograd` не было, то тогда backprop надо было бы реализовывать самим, и как это бы выглядело?

Рассмотрим на примере, как посчиать градиенты для весов из входного слоя, где входной вектора `X` состоит из 3-х компонент. А входной слой вторую размерность имеет равной 2. 

После чего это идет в `ReLU`, но для простоты опустим на время ее, и посмотрим как дальше это идет по сети.

Ниже написано, как это все вычисляется и приводит нас к значению целевой функции для одного наблюдения

<img src="./images/1.png" alt="1" style="width: 600px;"/>

Тогда, чтобы посчитать градиент по первому элементу из обучаемой матрицы на первом слое, необходимо взять производоную у сложной функции. А этот как раз делается по `chain rule`: сначала берем у внешней, потом спускаемся на уровень ниже, и так пока не додйдем до то функции, после которой эта перменная уже нигде не участвует:

<img src="./images/2.png" alt="2" style="width: 400px;"/>

Перепишем это все в матричном виде, то есть сделаем аналог вида матрицы весов из первого слоя, но там уже будут её градиенты, котоыре будут нужны чтобы как раз обновить эти веса:

<img src="./images/3.jpg" alt="3" style="width: 600px;"/>

Как видно, здесь можно вектор X вынести, то есть разделить на две матрицы:

<img src="./images/4.jpg" alt="4" style="width: 500px;"/>

То есть уже видно, что будем траспонировать входной вектор(матрицу). Но надо понимать, что в реальности у нас не одно наблюдение в батче, а несколько, тогда запись немного изменит свой вид:

<img src="./images/5.jpg" alt="5" style="width: 500px;"/>

Теперь мы видим, как на самом деле вычисляется вот те самые частные производные для вектора X, то есть видно, как математически это можно записать, а именно:

<img src="./images/6.jpg" alt="6" style="width: 500px;"/>

<img src="./images/7.jpg" alt="7" style="width: 500px;"/>

Уже можно реализовать. Понятно, что транспонируется, что нет, и что на что умножается.

Но помним про ReLU. Для простоты опустили, но теперь её учесть будет легче. 

Так как после первого слоя идет ReLU, а значит, занулились те выходы первого слоя, которые были __меньше__ нуля. Получается, что во второй слой не все дошло, тогда нужно обнулить, что занулил ReLU. 

Что занулил ReLU, мы можем выяснить при `forward pass`, а где именно поставить нули, то надо уже смотреть относительно `backward propagation`, на том выходе, где последний раз участвовал выход после ReLU, то есть:

<img src="./images/8.jpg" alt="8" style="width: 600px;"/>

Благодаря `Autograd` реализацию `chain rule` можно избежать, так как для более сложных нейронных сетей вручную такое реализовать сложно, при этом сделать это эффективным.

Для того чтобы PyTorch понял, за какими переменными надо "следить", то есть указать, что именно "эти" переменные являются обучаемыми, необходимо при создании тензора в качестве аттрибута указать __requires_grad=True__:

In [30]:
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)

In [31]:
learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Теперь подсчет градиентов для весов происходит при вызове backward
    loss.backward()
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # Теперь обнуляем значение градиентов, чтобы на следующем шаге 
        # они не учитывались при подсчете новых градиентов,
        # иначе произойдет суммирвоание старых и новых градиентов
        w1.grad.zero_()
        w2.grad.zero_()

99 74.82247161865234
199 74.29219055175781
299 73.80789184570312
399 73.3653793334961
499 72.96089935302734


Осталось еще не вручную обновлять веса, а использовать адаптивные методы градинетного спсука. Для этого нужно использовать модуль __optim__. А помимо оптимайзера, еще можно использовать готовые целевые функции из модлуя __nn__.

In [32]:
import torch.optim as optim

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
optimizer = torch.optim.Adam([w1, w2], lr=learning_rate)

for t in range(500):
    optimizer.zero_grad()
    
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    loss.backward()
   
    optimizer.step()

99 72.94839477539062
199 72.93968963623047
299 72.93099212646484
399 72.92241668701172
499 72.91392517089844


После того, как мы сделали backward, в этот момент посчитались градиенты и граф уничтожился, то есть стёрлись все пути, которые связывали тензоры между собой. Это значит, что еще раз backward сделать не поулчится, будет ошибка. Но если вдруг нужно считать градиенты еще раз, то нужно при вызове backward задать `retain_graph=True`.

Еще важный аттрибут, который есть у Tensor -- это `grad_fn`. В этом аттрибуте указывается та функция, посредством которой был создан этот тензор. Так PyTorch понимает, как именно считать по нему градиент.

In [33]:
y_pred.grad_fn

Также можно контролировать, должны ли градиенты течь или нет.

In [37]:
x = torch.tensor([1.], requires_grad=True)
with torch.no_grad():
    with torch.enable_grad():
        y = x * 2
y.requires_grad

True

## Почему Backprop надо понимать

1. Backprop позволяет понимать, как те или иные операции, сложные конструкции в сети влияют на обнолвение весов.
Почему лучше сделать конкатенацию тензоров, а не поэлементное сложение. Для этого нужно посмотреть на backprop, как будут обновляться веса.

2. Даже на таком маленьком пример двуслойной MLP можно уже увидеть, когда `ReLU`, как функция активация, не очень хорошо применять. Если разреженные данные, то получить на выходе много нулей вероятнее, чем при использовании `LeakyReLU`, то есть градиенты будут нулевыми и веса никак не будут обновляться => сеть не обучается!

3. В архитектуре могут встречаться недифференцируемые операции, и первое - это нужно понять, потому что при обучении сети это может быть не сразу заметно, просто качество модели будет плохое, и точность хорошую не поулчится достичь.

Например, в одной из статей было предложено в качестве механизма внимания применить распределение Бернулли, которое умножается на выход промежуточного слоя сети. И эта опреация недифференцируема, нужно реализовывать backprop самим, тем самым обеспечить корректное протекание градиентов.


<img src="./images/Bernoulli.png" alt="8" style="width: 600px;"/>

Так же любая статья, которая предлагет новую целевую функцию для той или иной задачи, там всегда будут представлены градиенты, чтобы было понимание, как это влияет на обновление весов. Не просто так !

<img src="./images/BernoulliBackProp.png" alt="8" style="width: 600px;"/>

## nn.Module

В предыдущем примере архитектуру сети создавали используя последовательной способ объявления слоев сети -- `nn.Sequential`.

Но еще можно это сделать более гибким подходом:

In [ ]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        TwoLayerNet наследуется от nn.Module и тем самым полчаем возможность
        переопределять методы класса.
        В конструктуре создаем слои (обучаемые веса) и другие нужные перменные/функции,
        которые нужны для модели
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        Метод forward отвечает за прямое распростронение модели, 
        поэтому данный метод нужно переопределять обязательно, 
        чтобы задать логику прямого распростронения. 
        Именно в этот момент начинает строится динамический граф
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        
        return y_pred

In [ ]:
batch_size = 64
input_size = 1000
hidden_size = 100
output_size = 10

x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

model = TwoLayerNet(Dinput_size_in, hidden_size, output_size)

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Домашние задание

1. Добавить Bias и посчитать для них градиенты.
2. Сравнить градинеты с тем, как считает PyTorch AutoGrad.

Рассчитаем градиенты:

$h_1 = x \cdot w_1 + b_1$

$h_r = ReLU(h_1)$

$L = (y - out)^2$

Для Loss:

$ \frac{\partial L} {\partial out}  = 2 \cdot (out - y)$

Для $w_2$ и $b_2$:

$ \frac {\partial L} {\partial w_2} = \frac {\partial L} {\partial out} \cdot \frac {\partial out} {\partial w_2} = 2 \cdot (out - y) \cdot h_r^T$

$ \frac {\partial L} {\partial b_2} = \frac {\partial L} {\partial out} \cdot \frac {\partial out} {\partial b_2} = 2 \cdot (out - y) \cdot 1 $ &ndash; так как производная по $b_2$ = $ (const) + \textcolor{red}{b_2} = 1$

Для $w_1$ и $b_1$:

$\frac {\partial L} {\partial w_1} = \frac {\partial L} {\partial out} \cdot \frac {\partial out} {\partial h_r} \cdot \frac {\partial h_r} {\partial h_1} \cdot \frac {\partial h_1} {\partial w_1} = 2 \cdot (out - y) \cdot \frac {\partial out} {\partial h_r} \cdot \frac {\partial h_r} {\partial h_1} \cdot x^T $

$\frac {\partial L} {\partial w_1} = \frac {\partial L} {\partial out} \cdot \frac {\partial out} {\partial h_r} \cdot \frac {\partial h_r} {\partial h_1} \cdot \frac {\partial h_1} {\partial w_1} = 2 \cdot (out - y) \cdot \frac {\partial out} {\partial h_r} \cdot \frac {\partial h_r} {\partial h_1} \cdot 1 $ &ndash; так как производная по $b_1$ тоже = $(const) + \textcolor{red}{b_1} = 1$

In [ ]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

*Извините за дурацкую инициализацию весов, ничего лучше не придумалось.....*

In [39]:
# Common random weights
w1_common = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
b1_common = torch.randn(batch_size, hidden_size, device=device, dtype=dtype)
w2_common = torch.randn(hidden_size, output_size, device=device, dtype=dtype)
b2_common = torch.randn(batch_size, output_size, device=device, dtype=dtype)

In [40]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights
w1 = w1_common.clone()
b1 = b1_common.clone()
w2 = w2_common.clone()
b2 = b2_common.clone()

learning_rate = 1e-6
for t in range(500):
    h_1 = x.mm(w1) + b1
    h_relu = h_1.clamp(min=0)
    out = h_relu.mm(w2) + b2

    loss = (out - y).pow(2).sum().item() #MSE

    dloss_dout = 2 * (out - y)  # первая производная по лоссу

    grad_w2 = h_relu.t().mm(dloss_dout)
    grad_b2 = dloss_dout
    grad_hrelu = dloss_dout.mm(w2.t())
    grad_hrelu[h_1 < 0] = 0
    grad_w1 = x.t().mm(grad_hrelu)
    grad_b1 = grad_hrelu
 
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    b1 -= learning_rate * grad_b1
    b2 -= learning_rate * grad_b2
    if t % 100 == 99:
        print('Loss on iteration {} = {}'.format(t, loss))
    

Loss on iteration 99 = 132.50535583496094
Loss on iteration 199 = 130.90997314453125
Loss on iteration 299 = 129.5211639404297
Loss on iteration 399 = 128.3069610595703
Loss on iteration 499 = 127.24038696289062


### PyTorch AutoGrad

In [46]:
w1 = w1_common.clone()
b1 = b1_common.clone()
w2 = w2_common.clone()
b2 = b2_common.clone()
w1.requires_grad=True
b1.requires_grad=True
w2.requires_grad=True
b2.requires_grad=True

In [47]:
import torch.optim as optim

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
optimizer = torch.optim.SGD([w1, w2, b1, b2], lr=learning_rate)

for t in range(500):
    optimizer.zero_grad()
    
    y_pred = (x.mm(w1) + b1).clamp(min=0).mm(w2) + b2
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    loss.backward()
   
    optimizer.step()

99 132.50535583496094
199 130.90997314453125
299 129.5211639404297
399 128.3069610595703
499 127.24038696289062
